In [1]:
import torch
import src
import wandb
import numpy as np
import cv2
import os

from PIL import Image
from src.utils.dataset import Preprocessing
from src.utils.dataset import Dataset
from sklearn.model_selection import train_test_split

In [2]:
ENCODER = 'resnet152'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu') 

In [3]:
model = src.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=1,
    classes=1,
    activation=ACTIVATION,
)

In [4]:
loss = src.utils.base.SumOfLosses(
    src.utils.losses.DiceLoss(),
    src.utils.losses.BCELoss()
)

metrics = [
    src.utils.metrics.IoU(threshold=0.5)
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
        factor=0.01, patience=10, threshold=0.0001, threshold_mode='abs')

In [5]:
train_epoch = src.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = src.utils.train.ValidEpoch(
    model,
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [6]:
preprocess = Preprocessing(
    dir='./data/',
    img_size=768,
    augmentation=False
)

In [7]:
wandb.login()

wandb.init(project="Trus_images")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dalyutkin01. Use `wandb login --relogin` to force relogin


In [8]:
max_IoU = 0.43
clear_images, clear_masks = preprocess.load_folder()
X_train, X_test, y_train, y_test = train_test_split(clear_images, clear_masks, test_size=0.2, random_state=42)

In [9]:
for i in range(0, 1):
    train_loader, valid_loader = preprocess.Generator(
        batch_size=2, 
        X_train=X_train, 
        X_test=X_test, 
        y_train=y_train, 
        y_test=y_test)
    print('\nEpoch: {}'.format(i + 1))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    wandb.log({'train/train_IoU': train_logs['iou_score'], 'train/train_loss': train_logs['dice_loss + bce_loss']})
    wandb.log({'valid/valid_IoU': valid_logs['iou_score'], 'valid/valid_loss': valid_logs['dice_loss + bce_loss']})
    scheduler.step(valid_logs['dice_loss + bce_loss'])
    if max_IoU < valid_logs['iou_score']:
        max_IoU = valid_logs['iou_score']
        torch.save({
            'model_state_dict': model.state_dict(),
            'best_IoU' : max_IoU
            }, './checkpoint/best_model_without_kfold.pth')
        print('Model saved!')
        
wandb.finish()


Epoch: 1
valid: 100%|██████████| 11/11 [00:10<00:00,  1.08it/s, dice_loss + bce_loss - 0.5842, iou_score - 0.3253]


train/train_IoU,▁
train/train_loss,▁
valid/valid_IoU,▁
valid/valid_loss,▁
train/train_IoU,0.18166
train/train_loss,0.78436
valid/valid_IoU,0.32531
valid/valid_loss,0.58421


In [10]:
checkpoint = torch.load('./checkpoint/best_model_without_kfold.pth')

In [11]:
checkpoint['best_IoU']

0.44076838547533204

In [12]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [13]:
import os
import cv2
from PIL import Image

In [14]:
shape = X_test[0].shape[:2]

In [15]:
preds = []
for image in X_test:
    image_prepoc = preprocess.prepocessing_image(image)
    image_prepoc = torch.tensor(np.array([[image_prepoc]]), dtype = torch.float)
    image_prepoc = image_prepoc.to(DEVICE)
    with torch.no_grad():
        predict = model.forward(image_prepoc)
    predict_value = predict.cpu().detach().numpy()
    preds.append(predict_value)

In [16]:
valid_pred_folder = './valid_predict_v1/'

In [17]:
def preproc_predict(image):
    predict = (image[0][0]*255).astype(np.uint8)
    img  = np.array(Image.fromarray(predict))
    return img

In [18]:
def green_chanel(image):
    blue,green,red = cv2.split(image)
    zeros = np.zeros(blue.shape, np.uint8)
    greenBGR = cv2.merge([zeros,green,zeros])
    return greenBGR

In [19]:
def blue_chanel_predict(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1280, 872), interpolation = cv2.INTER_AREA)
    blue, green, _ = cv2.split(image)
    zeros = np.zeros(blue.shape, np.uint8)
    blueBGR = cv2.merge([blue,zeros,zeros])
    return blueBGR

def green_chanel_predict(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1280, 872), interpolation = cv2.INTER_AREA)
    blue, green, _ = cv2.split(image)
    zeros = np.zeros(blue.shape, np.uint8)
    green = cv2.merge([zeros,green,zeros])
    return green

In [20]:
def calc_IoU(image_first, image_second):
    bitwiseOR = cv2.bitwise_or(image_first, image_second)
    bitwiseOR = bitwiseOR / 255
    bitwiseOR = (np.rint(bitwiseOR)).astype(int)
    summary = bitwiseOR.sum()
    bitwiseAnd = cv2.bitwise_and(image_first, image_second)
    bitwiseAnd = bitwiseAnd / 255
    bitwiseAnd = (np.rint(bitwiseAnd)).astype(int)
    peres = bitwiseAnd.sum()
    return peres / summary

In [21]:
for num in range(len(X_test)):
    true_mask = y_test[num]
    greenBGR = green_chanel(true_mask)
    concat_true = cv2.addWeighted(X_test[num], 0.4, y_test[num], 0.9, 0)
    
    #resize true image and create 3rd channels
    cvtColor = cv2.cvtColor(X_test[num], cv2.COLOR_BGR2GRAY )
    resize_image = cv2.resize(cvtColor, (1280, 872), interpolation = cv2.INTER_AREA)
    resize_image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2RGB)
    
    #preproc predict
    predict_mask = preproc_predict(preds[num])
    blueBGR_predict = blue_chanel_predict(predict_mask)
    greenBGR_predict = green_chanel_predict(predict_mask)
    # concat_predict = cv2.bitwise_or(resize_image, predict_mask, mask = predict_mask)
    # image_rgb=cv2.cvtColor(concat_predict, cv2.COLOR_BGR2RGB)
    
    #calc IoU
    # print(greenBGR.shape)
    # print(greenBGR_predict.shape)
    IoU = calc_IoU(greenBGR, greenBGR_predict)
    
    #concat two masks
    bitwiseAnd = cv2.bitwise_or(greenBGR, blueBGR_predict)
    
    #concat true images with mask
    concat_predict = cv2.addWeighted(resize_image, 0.4, bitwiseAnd, 0.7, 0)
    concat_predict_resize = cv2.resize(concat_predict, (1280, 872), interpolation = cv2.INTER_AREA)
    
    #put text
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (570, 50)
    fontScale = 1
    color = (0, 255, 255)
    thickness = 2
    image = cv2.putText(concat_predict_resize, 'IoU ' + str(round(IoU, 3)), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
    
    #save 
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_true" + '.png'), img=concat_true)
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_predict" + '.png'), img=concat_predict_resize)